In [6]:
import copy
import os.path
import sys

In [7]:
os.getcwd()

'/home/minwoo/LESN/DecomposeBERT'

In [8]:
sys.path.append('/home/minwoo/LESN/DecomposeBERT/')

In [9]:
from utils.model_utils.evaluate import evaluate_model, test_f1, test

from utils.model_utils.load_model import *
from utils.model_utils.model_config import ModelConfig
from utils.dataset_utils.load_dataset import load_dataset
from utils.decompose_utils.weight_remover import WeightRemoverBert
from utils.decompose_utils.concern_identification import ConcernIdentificationBert
from utils.decompose_utils.tangling_identification import TanglingIdentification
from transformers import AutoConfig
from tqdm import tqdm
from datetime import datetime
from pprint import pp
from utils.decompose_utils.sampling import sampling_class
import torch

In [10]:
# model_name = "prajjwal1/bert-tiny"
# model_dir = "tiny bert"
model_name = "bert-base-uncased"
model_dir = "bert"
data = "SDG"
num_labels = 16
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
checkpoint_name = "best_model.pt"
config = AutoConfig.from_pretrained(model_name, num_labels=num_labels)
model_config = ModelConfig(
    _model_name=model_name,
    _model_dir=model_dir,
    _data=data,
    _is_pretrained=True,
    _transformer_config=config,
    _checkpoint_name=checkpoint_name,
    _device=device,
)
model, tokenizer, checkpoint = load_classification_model(model_config)

train_dataloader, valid_dataloader, test_dataloader = load_dataset(
    model_config, tokenizer, batch_size=32, test_size=0.3
)
# print("Start Time:" + datetime.now().strftime("%H:%M:%S"))
# evaluate_model(model, model_config, test_dataloader)

Loading dataset
Loading preprocessed data...
Loading preprocessed data...
Loading preprocessed data...


In [11]:
for k in range(3):
    i = 9
    print("#Module " + str(i) + " in progress....")
    num_samples = 64

    positive_samples1 = sampling_class(
        train_dataloader, i, 20, num_labels, True, 4, device=device
    )
    positive_samples2 = sampling_class(
        train_dataloader, i, num_samples, num_labels, True, 4, device=device
    )
    negative_samples = sampling_class(
        train_dataloader, i, num_samples, num_labels, False, 4, device=device
    )

    module1 = copy.deepcopy(model)
    w = WeightRemoverBert(model, p=0.9)
    ci1 = ConcernIdentificationBert(model, p=0.5)
    ti1 = TanglingIdentification(model, p=0.6)

    ff1 = [
        [
            torch.sum(
                model.bert.encoder.layer[num].intermediate.dense.weight != 0
            ).item()
        ]
        for num in range(config.num_hidden_layers)
    ]
    ff2 = [
        [torch.sum(model.bert.encoder.layer[num].output.dense.weight != 0).item()]
        for num in range(config.num_hidden_layers)
    ]
    pooler = [torch.sum(model.bert.pooler.dense.weight != 0).item()]
    print("origin")
    j = 0
    print(j)
    results = test_f1(model, test_dataloader, model_config, False)
    pp(results["details"][f"{i}"])

    print("Start Positive CI sparse")

    for batch in positive_samples1:
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            t1 = w.propagate(module1, input_ids)
        for num in range(config.num_hidden_layers):
            ff1[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].intermediate.dense.weight != 0
                ).item()
            )
            ff2[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].output.dense.weight != 0
                ).item()
            )
        pooler.append(torch.sum(module1.bert.pooler.dense.weight != 0).item())

        j+=1
        print(j)
        
        results = test_f1(module1, test_dataloader, model_config, False)
        pp(results["details"][f"{i}"])        

    print("Start Positive CI after sparse")

    for batch in positive_samples2:
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            t1 = ci1.propagate(module1, input_ids)
        for num in range(config.num_hidden_layers):
            ff1[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].intermediate.dense.weight != 0
                ).item()
            )
            ff2[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].output.dense.weight != 0
                ).item()
            )
        pooler.append(torch.sum(module1.bert.pooler.dense.weight != 0).item())

        j+=1
        print(j)
        
        results = test_f1(module1, test_dataloader, model_config, False)
        pp(results["details"][f"{i}"])        

    print("Start Negative TI")

    for batch in negative_samples:
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            t = ti1.propagate(module1, input_ids)
        for num in range(config.num_hidden_layers):
            ff1[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].intermediate.dense.weight != 0
                ).item()
            )
            ff2[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].output.dense.weight != 0
                ).item()
            )
        pooler.append(torch.sum(module1.bert.pooler.dense.weight != 0).item())

        j+=1
        print(j)
        
        results = test_f1(module1, test_dataloader, model_config, False)
        pp(results["details"][f"{i}"])        

    for num in range(config.num_hidden_layers):
        print(f"f({ff1[num]})")
        print("\n")
    for num in range(config.num_hidden_layers):
        print(f"f({ff2[num]})")
        print("\n")
    print(f"f({pooler})")
    print("\n")

#Module 9 in progress....
origin
0
{'precision': 0.5849056603773585,
 'recall': 0.6919642857142857,
 'f1-score': 0.6339468302658486,
 'support': 224.0}
Start Positive CI sparse
1
{'precision': 0.5740740740740741,
 'recall': 0.6919642857142857,
 'f1-score': 0.6275303643724697,
 'support': 224.0}
2
{'precision': 0.5735849056603773,
 'recall': 0.6785714285714286,
 'f1-score': 0.6216768916155418,
 'support': 224.0}
3
{'precision': 0.5735849056603773,
 'recall': 0.6785714285714286,
 'f1-score': 0.6216768916155418,
 'support': 224.0}
4
{'precision': 0.5735849056603773,
 'recall': 0.6785714285714286,
 'f1-score': 0.6216768916155418,
 'support': 224.0}
5
{'precision': 0.5735849056603773,
 'recall': 0.6785714285714286,
 'f1-score': 0.6216768916155418,
 'support': 224.0}
Start Positive CI after sparse
6
{'precision': 0.7304347826086957,
 'recall': 0.375,
 'f1-score': 0.495575221238938,
 'support': 224.0}
7
{'precision': 0.5205479452054794,
 'recall': 0.6785714285714286,
 'f1-score': 0.5891472868

In [12]:
for k in range(3):
    i = 10
    print("#Module " + str(i) + " in progress....")
    num_samples = 64

    positive_samples1 = sampling_class(
        train_dataloader, i, 20, num_labels, True, 4, device=device
    )
    positive_samples2 = sampling_class(
        train_dataloader, i, num_samples, num_labels, True, 4, device=device
    )
    negative_samples = sampling_class(
        train_dataloader, i, num_samples, num_labels, False, 4, device=device
    )

    module1 = copy.deepcopy(model)
    w = WeightRemoverBert(model, p=0.9)
    ci1 = ConcernIdentificationBert(model, p=0.5)
    ti1 = TanglingIdentification(model, p=0.6)

    ff1 = [
        [
            torch.sum(
                model.bert.encoder.layer[num].intermediate.dense.weight != 0
            ).item()
        ]
        for num in range(config.num_hidden_layers)
    ]
    ff2 = [
        [torch.sum(model.bert.encoder.layer[num].output.dense.weight != 0).item()]
        for num in range(config.num_hidden_layers)
    ]
    pooler = [torch.sum(model.bert.pooler.dense.weight != 0).item()]
    print("origin")
    j = 0
    print(j)
    results = test_f1(model, test_dataloader, model_config, False)
    pp(results["details"][f"{i}"])

    print("Start Positive CI sparse")

    for batch in positive_samples1:
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            t1 = w.propagate(module1, input_ids)
        for num in range(config.num_hidden_layers):
            ff1[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].intermediate.dense.weight != 0
                ).item()
            )
            ff2[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].output.dense.weight != 0
                ).item()
            )
        pooler.append(torch.sum(module1.bert.pooler.dense.weight != 0).item())

        j+=1
        print(j)
        
        results = test_f1(module1, test_dataloader, model_config, False)
        pp(results["details"][f"{i}"])        

    print("Start Positive CI after sparse")

    for batch in positive_samples2:
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            t1 = ci1.propagate(module1, input_ids)
        for num in range(config.num_hidden_layers):
            ff1[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].intermediate.dense.weight != 0
                ).item()
            )
            ff2[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].output.dense.weight != 0
                ).item()
            )
        pooler.append(torch.sum(module1.bert.pooler.dense.weight != 0).item())

        j+=1
        print(j)
        
        results = test_f1(module1, test_dataloader, model_config, False)
        pp(results["details"][f"{i}"])        

    print("Start Negative TI")

    for batch in negative_samples:
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            t = ti1.propagate(module1, input_ids)
        for num in range(config.num_hidden_layers):
            ff1[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].intermediate.dense.weight != 0
                ).item()
            )
            ff2[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].output.dense.weight != 0
                ).item()
            )
        pooler.append(torch.sum(module1.bert.pooler.dense.weight != 0).item())

        j+=1
        print(j)
        
        results = test_f1(module1, test_dataloader, model_config, False)
        pp(results["details"][f"{i}"])        

    for num in range(config.num_hidden_layers):
        print(f"f({ff1[num]})")
        print("\n")
    for num in range(config.num_hidden_layers):
        print(f"f({ff2[num]})")
        print("\n")
    print(f"f({pooler})")
    print("\n")

#Module 10 in progress....
origin
0
{'precision': 0.7760416666666666,
 'recall': 0.6563876651982379,
 'f1-score': 0.7112171837708832,
 'support': 227.0}
Start Positive CI sparse
1
{'precision': 0.7619047619047619,
 'recall': 0.6343612334801763,
 'f1-score': 0.6923076923076923,
 'support': 227.0}
2
{'precision': 0.7789473684210526,
 'recall': 0.6519823788546255,
 'f1-score': 0.709832134292566,
 'support': 227.0}
3
{'precision': 0.7789473684210526,
 'recall': 0.6519823788546255,
 'f1-score': 0.709832134292566,
 'support': 227.0}
4
{'precision': 0.7789473684210526,
 'recall': 0.6519823788546255,
 'f1-score': 0.709832134292566,
 'support': 227.0}
5
{'precision': 0.7789473684210526,
 'recall': 0.6519823788546255,
 'f1-score': 0.709832134292566,
 'support': 227.0}
Start Positive CI after sparse
6
{'precision': 0.5683760683760684,
 'recall': 0.5859030837004405,
 'f1-score': 0.5770065075921909,
 'support': 227.0}
7
{'precision': 0.7653631284916201,
 'recall': 0.6035242290748899,
 'f1-score': 0

In [13]:
for k in range(3):
    i = 11
    print("#Module " + str(i) + " in progress....")
    num_samples = 64

    positive_samples1 = sampling_class(
        train_dataloader, i, 20, num_labels, True, 4, device=device
    )
    positive_samples2 = sampling_class(
        train_dataloader, i, num_samples, num_labels, True, 4, device=device
    )
    negative_samples = sampling_class(
        train_dataloader, i, num_samples, num_labels, False, 4, device=device
    )

    module1 = copy.deepcopy(model)
    w = WeightRemoverBert(model, p=0.9)
    ci1 = ConcernIdentificationBert(model, p=0.5)
    ti1 = TanglingIdentification(model, p=0.6)

    ff1 = [
        [
            torch.sum(
                model.bert.encoder.layer[num].intermediate.dense.weight != 0
            ).item()
        ]
        for num in range(config.num_hidden_layers)
    ]
    ff2 = [
        [torch.sum(model.bert.encoder.layer[num].output.dense.weight != 0).item()]
        for num in range(config.num_hidden_layers)
    ]
    pooler = [torch.sum(model.bert.pooler.dense.weight != 0).item()]
    print("origin")
    j = 0
    print(j)
    results = test_f1(model, test_dataloader, model_config, False)
    pp(results["details"][f"{i}"])

    print("Start Positive CI sparse")

    for batch in positive_samples1:
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            t1 = w.propagate(module1, input_ids)
        for num in range(config.num_hidden_layers):
            ff1[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].intermediate.dense.weight != 0
                ).item()
            )
            ff2[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].output.dense.weight != 0
                ).item()
            )
        pooler.append(torch.sum(module1.bert.pooler.dense.weight != 0).item())

        j+=1
        print(j)
        
        results = test_f1(module1, test_dataloader, model_config, False)
        pp(results["details"][f"{i}"])        

    print("Start Positive CI after sparse")

    for batch in positive_samples2:
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            t1 = ci1.propagate(module1, input_ids)
        for num in range(config.num_hidden_layers):
            ff1[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].intermediate.dense.weight != 0
                ).item()
            )
            ff2[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].output.dense.weight != 0
                ).item()
            )
        pooler.append(torch.sum(module1.bert.pooler.dense.weight != 0).item())

        j+=1
        print(j)
        
        results = test_f1(module1, test_dataloader, model_config, False)
        pp(results["details"][f"{i}"])        

    print("Start Negative TI")

    for batch in negative_samples:
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            t = ti1.propagate(module1, input_ids)
        for num in range(config.num_hidden_layers):
            ff1[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].intermediate.dense.weight != 0
                ).item()
            )
            ff2[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].output.dense.weight != 0
                ).item()
            )
        pooler.append(torch.sum(module1.bert.pooler.dense.weight != 0).item())

        j+=1
        print(j)
        
        results = test_f1(module1, test_dataloader, model_config, False)
        pp(results["details"][f"{i}"])        

    for num in range(config.num_hidden_layers):
        print(f"f({ff1[num]})")
        print("\n")
    for num in range(config.num_hidden_layers):
        print(f"f({ff2[num]})")
        print("\n")
    print(f"f({pooler})")
    print("\n")

#Module 11 in progress....
origin
0
{'precision': 0.7961165048543689,
 'recall': 0.7387387387387387,
 'f1-score': 0.7663551401869159,
 'support': 111.0}
Start Positive CI sparse
1
{'precision': 0.7920792079207921,
 'recall': 0.7207207207207207,
 'f1-score': 0.7547169811320755,
 'support': 111.0}
2
{'precision': 0.8260869565217391,
 'recall': 0.6846846846846847,
 'f1-score': 0.7487684729064039,
 'support': 111.0}
3
{'precision': 0.8260869565217391,
 'recall': 0.6846846846846847,
 'f1-score': 0.7487684729064039,
 'support': 111.0}
4
{'precision': 0.8260869565217391,
 'recall': 0.6846846846846847,
 'f1-score': 0.7487684729064039,
 'support': 111.0}
5
{'precision': 0.8260869565217391,
 'recall': 0.6846846846846847,
 'f1-score': 0.7487684729064039,
 'support': 111.0}
Start Positive CI after sparse
6
{'precision': 1.0,
 'recall': 0.16216216216216217,
 'f1-score': 0.27906976744186046,
 'support': 111.0}
7
{'precision': 0.8888888888888888,
 'recall': 0.5765765765765766,
 'f1-score': 0.69945355

In [14]:
for k in range(3):
    i = 12
    print("#Module " + str(i) + " in progress....")
    num_samples = 64

    positive_samples1 = sampling_class(
        train_dataloader, i, 20, num_labels, True, 4, device=device
    )
    positive_samples2 = sampling_class(
        train_dataloader, i, num_samples, num_labels, True, 4, device=device
    )
    negative_samples = sampling_class(
        train_dataloader, i, num_samples, num_labels, False, 4, device=device
    )

    module1 = copy.deepcopy(model)
    w = WeightRemoverBert(model, p=0.9)
    ci1 = ConcernIdentificationBert(model, p=0.5)
    ti1 = TanglingIdentification(model, p=0.6)

    ff1 = [
        [
            torch.sum(
                model.bert.encoder.layer[num].intermediate.dense.weight != 0
            ).item()
        ]
        for num in range(config.num_hidden_layers)
    ]
    ff2 = [
        [torch.sum(model.bert.encoder.layer[num].output.dense.weight != 0).item()]
        for num in range(config.num_hidden_layers)
    ]
    pooler = [torch.sum(model.bert.pooler.dense.weight != 0).item()]
    print("origin")
    j = 0
    print(j)
    results = test_f1(model, test_dataloader, model_config, False)
    pp(results["details"][f"{i}"])

    print("Start Positive CI sparse")

    for batch in positive_samples1:
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            t1 = w.propagate(module1, input_ids)
        for num in range(config.num_hidden_layers):
            ff1[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].intermediate.dense.weight != 0
                ).item()
            )
            ff2[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].output.dense.weight != 0
                ).item()
            )
        pooler.append(torch.sum(module1.bert.pooler.dense.weight != 0).item())

        j+=1
        print(j)
        
        results = test_f1(module1, test_dataloader, model_config, False)
        pp(results["details"][f"{i}"])        

    print("Start Positive CI after sparse")

    for batch in positive_samples2:
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            t1 = ci1.propagate(module1, input_ids)
        for num in range(config.num_hidden_layers):
            ff1[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].intermediate.dense.weight != 0
                ).item()
            )
            ff2[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].output.dense.weight != 0
                ).item()
            )
        pooler.append(torch.sum(module1.bert.pooler.dense.weight != 0).item())

        j+=1
        print(j)
        
        results = test_f1(module1, test_dataloader, model_config, False)
        pp(results["details"][f"{i}"])        

    print("Start Negative TI")

    for batch in negative_samples:
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            t = ti1.propagate(module1, input_ids)
        for num in range(config.num_hidden_layers):
            ff1[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].intermediate.dense.weight != 0
                ).item()
            )
            ff2[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].output.dense.weight != 0
                ).item()
            )
        pooler.append(torch.sum(module1.bert.pooler.dense.weight != 0).item())

        j+=1
        print(j)
        
        results = test_f1(module1, test_dataloader, model_config, False)
        pp(results["details"][f"{i}"])        

    for num in range(config.num_hidden_layers):
        print(f"f({ff1[num]})")
        print("\n")
    for num in range(config.num_hidden_layers):
        print(f"f({ff2[num]})")
        print("\n")
    print(f"f({pooler})")
    print("\n")

#Module 12 in progress....
origin
0
{'precision': 0.8181818181818182,
 'recall': 0.7285714285714285,
 'f1-score': 0.7707808564231738,
 'support': 210.0}
Start Positive CI sparse
1
{'precision': 0.7586206896551724,
 'recall': 0.7333333333333333,
 'f1-score': 0.7457627118644068,
 'support': 210.0}
2
{'precision': 0.7323943661971831,
 'recall': 0.7428571428571429,
 'f1-score': 0.7375886524822696,
 'support': 210.0}
3
{'precision': 0.7323943661971831,
 'recall': 0.7428571428571429,
 'f1-score': 0.7375886524822696,
 'support': 210.0}
4
{'precision': 0.7323943661971831,
 'recall': 0.7428571428571429,
 'f1-score': 0.7375886524822696,
 'support': 210.0}
5
{'precision': 0.7323943661971831,
 'recall': 0.7428571428571429,
 'f1-score': 0.7375886524822696,
 'support': 210.0}
Start Positive CI after sparse
6
{'precision': 0.926605504587156,
 'recall': 0.48095238095238096,
 'f1-score': 0.6332288401253918,
 'support': 210.0}
7
{'precision': 0.7634408602150538,
 'recall': 0.6761904761904762,
 'f1-score